In [158]:
import pandas as pd
import os
import re

In [160]:
file_paths = [
    r"E:\Sales closing Dump\2021-22\Daily sales original Feb22 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\March22 Closing Dump (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Apr21 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Aug21.xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Dec21.xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Jan22 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Jul21.xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Jun21 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR May21 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Nov21 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Oct21 (1).xlsx",
    r"E:\Sales closing Dump\2021-22\PCR Sep21 (1).xlsx"
]

In [162]:
# The rest of the code remains unchanged
compiled_data = pd.DataFrame()

for file_path in file_paths:
    data = pd.read_excel(file_path)
    compiled_data = pd.concat([compiled_data, data], ignore_index=True)


In [163]:
compiled_data.columns

Index(['Month', 'Zone', 'Region', 'Area', 'Account Group', 'Account Group.1',
       'Bill Type', 'Classification', 'Classification.1', 'Customer',
       'Customer.1', 'Inches', 'Material', 'Material.1', 'Pattern', 'Total'],
      dtype='object')

In [164]:
def determine_product_type(value_or_material):
    # First, handle the PCR-BIAS and PCR-RADIAL conditions (from the first function)
    if value_or_material.startswith("131") or value_or_material.startswith("133") or value_or_material.startswith("141") or value_or_material.startswith("143"):
        return "PCR-BIAS"
    elif value_or_material.startswith("132") or value_or_material.startswith("142") or value_or_material.startswith("145"):
        return "PCR-RADIAL"
    elif value_or_material.startswith("121") or value_or_material.startswith("123"):
        return "LCV-BIAS"
    elif value_or_material.startswith("122") or value_or_material.startswith("125"):
        return "LCV-RADIAL"
    elif value_or_material.startswith("1D1") or value_or_material.startswith("1D3"):
        return "SCV-BIAS"
    elif value_or_material.startswith("1D2") or value_or_material.startswith("1D5"):
        return "SCV-RADIAL"
    elif value_or_material.startswith("1S"):
        return "SCOOTER"
    elif value_or_material.startswith("1B"):
        return "MOTORCYCLE"
    elif value_or_material.startswith("1H"):
        return "3W"
    elif value_or_material.startswith("111") or value_or_material.startswith("113"):
        return "TRUCK-BIAS"
    elif value_or_material.startswith("112") or value_or_material.startswith("115"):
        return "TRUCK-RADIAL"
    elif value_or_material.startswith("1E1") or value_or_material.startswith("181"):
        return "OTR"
    elif value_or_material.startswith("15") or value_or_material.startswith("16") or value_or_material.startswith("17") or value_or_material.startswith("19"):
        return "FARM"
    else:
        return "NA"


In [165]:
compiled_data['Product_type'] = compiled_data['Material'].apply(determine_product_type)

In [166]:
compiled_data['Month'].value_counts()

Month
MAR 2022    119110
FEB 2022    109586
AUG 2021     42191
JUL 2021     41882
DEC 2021     41535
OCT 2021     41062
NOV 2021     40247
SEP 2021     38892
JAN 2022     35999
JUN 2021     34957
APR 2021     30067
MAY 2021     17427
Name: count, dtype: int64

In [167]:
def assign_quarter(month):
    # Convert month to lowercase to make comparison case-insensitive
    month_lower = month.lower()
    
    # Define the month-year combinations and their respective quarters
    quarter_mapping = {
        "jan 2022": "Q4",
        "feb 2022": "Q4",
        "mar 2022": "Q4",
        "apr 2021": "Q1",
        "may 2021": "Q1",
        "jun 2021": "Q1",
        "jul 2021": "Q2",
        "aug 2021": "Q2",
        "sep 2021": "Q2",
        "oct 2021": "Q3",
        "nov 2021": "Q3",
        "dec 2021": "Q3",
    }
    
    # Return the quarter if it exists in the mapping, else return "Unknown"
    return quarter_mapping.get(month_lower, "Unknown")


In [168]:
compiled_data['Quarter'] = compiled_data['Month'].apply(assign_quarter)


In [169]:
compiled_data.columns

Index(['Month', 'Zone', 'Region', 'Area', 'Account Group', 'Account Group.1',
       'Bill Type', 'Classification', 'Classification.1', 'Customer',
       'Customer.1', 'Inches', 'Material', 'Material.1', 'Pattern', 'Total',
       'Product_type', 'Quarter'],
      dtype='object')

In [170]:
df = pd.read_excel(r"E:\Sales closing Dump\2021-22\SAp id in score card.xlsx")

In [171]:
compiled_data['Exists'] = compiled_data['Customer'].isin(df['SAP ID']).map({True: 'Yes', False: 'No'})


In [208]:
filtered_df = compiled_data[
    (compiled_data['Exists'] == 'Yes') &
    (compiled_data['Product_type'] == 'PCR-RADIAL') &
    ((compiled_data['Account Group'] == 'Z001') | 
     ((compiled_data['Account Group'] == 'Z004') & compiled_data['Classification.1'].isin(['DB', 'RD','DISTRIBUTOR','Rural Distribution']))) &
    (compiled_data['Bill Type'].isin(['ZOR', 'YOR', 'YPLT', 'ZPLT', 'ZRDR', 'YRDR','Depot Invoice','Depot Returns','Plant Invoice']))
]

pivot_table = filtered_df.pivot_table(
    index='Quarter',
    values=['Customer', 'Total'],
    aggfunc={'Customer': 'count', 'Total': 'sum'},
    fill_value=0
)

print(pivot_table)


         Customer   Total
Quarter                  
Q1          67772  466495
Q2          94530  681620
Q3          94810  669755
Q4          81866  545429


In [204]:
df3 = compiled_data[compiled_data['Quarter'] == 'Q4'].copy()

In [214]:
filtered_df_all= compiled_data[
     (compiled_data['Product_type'] == 'PCR-RADIAL') &
    ((compiled_data['Account Group'] == 'Z001') | 
     ((compiled_data['Account Group'] == 'Z004') & compiled_data['Classification.1'].isin(['DB', 'RD','DISTRIBUTOR','Rural Distribution'])))
]

pivot_table_all= filtered_df_all.pivot_table(
    index='Quarter',
    values=['Customer', 'Total'],
    aggfunc={'Customer': 'count', 'Total': 'sum'},
    fill_value=0
)

print(pivot_table_all)


         Customer   Total
Quarter                  
Q1          81132  537233
Q2         121079  802783
Q3         120900  786373
Q4         101951  613510


In [206]:
df3.to_csv(r"E:\Sales closing Dump\2021-22\Q4.csv")